<a href="https://colab.research.google.com/github/kc2409/cl1/blob/main/paper_multi_perspective.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_seq_length = 128


class StanceDetectionModel(nn.Module):
    def __init__(self, bert_model):
        super(StanceDetectionModel, self).__init__()
        self.bert = bert_model
        self.stance_classifier = nn.Linear(bert_model.config.hidden_size, num_stance_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        logits = self.stance_classifier(last_hidden_state[:, 0, :])
        return logits

bert_model = BertModel.from_pretrained("bert-base-uncased")
stance_model = StanceDetectionModel(bert_model)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(stance_model.parameters(), lr=learning_rate)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    stance_model.train()
    total_loss = 0.0

    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        logits = stance_model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    #
    average_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}")

# Inference
#stance_model.eval()
#with torch.no_grad():
#    for batch in test_dataloader:
#        input_ids, attention_mask = batch
#        logits = stance_model(input_ids, attention_mask)
#        predicted_labels = torch.argmax(logits, dim=1)



In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the projection layers
class ProjectionLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ProjectionLayer, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

# Define the Target-oriented contrastive loss
class TargetContrastiveLoss(nn.Module):
    def __init__(self, temperature):
        super(TargetContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, anchor, positive, negative):
        # Calculate the contrastive loss for target
        target_similarity = torch.nn.functional.cosine_similarity(anchor, positive, dim=-1) / self.temperature
        loss = -torch.log(torch.exp(target_similarity).sum() / (torch.exp(target_similarity).sum() + torch.exp(negative).sum()))
        return loss

# Define the Label-oriented contrastive loss
class LabelContrastiveLoss(nn.Module):
    def __init__(self, temperature):
        super(LabelContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, anchor, positive, negative):
        # Calculate the contrastive loss for label
        label_similarity = torch.nn.functional.cosine_similarity(anchor, positive, dim=-1) / self.temperature
        loss = -torch.log(torch.exp(label_similarity).sum() / (torch.exp(label_similarity).sum() + torch.exp(negative).sum()))
        return loss

# Define your StanceModel class
class StanceModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, temperature):
        super(StanceModel, self).__init__()  # Add parentheses here
        self.target_projection = ProjectionLayer(input_dim, hidden_dim, output_dim)
        self.text_projection = ProjectionLayer(input_dim, hidden_dim, output_dim)
        self.label_projection = ProjectionLayer(input_dim, hidden_dim, output_dim)
        self.target_contrastive_loss = TargetContrastiveLoss(temperature)
        self.label_contrastive_loss = LabelContrastiveLoss(temperature)

    def forward(self, target, text, label):
        target_projection = self.target_projection(target)
        text_projection = self.text_projection(text)
        label_projection = self.label_projection(label)

        return target_projection, text_projection, label_projection

input_dim = 300  # If you're using 300-dimensional word embeddings (example value)
hidden_dim = 256  # Adjust based on your model's complexity (example value)
output_dim = 3  # Since it appears you have three classes in your 'stance_label' column
temperature = 1.0  # A common starting value for contrastive learning

# Instantiate the StanceModel with these hyperparameters
model = StanceModel(input_dim, hidden_dim, output_dim, temperature)



# Instantiate the StanceModel, optimizer, and contrastive losses
model = StanceModel(input_dim, hidden_dim, output_dim, temperature)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        targets, texts, labels = batch

        # Forward pass
        target_proj, text_proj, label_proj = model(targets, texts, labels)

        # Calculate target contrastive loss and label contrastive loss
        target_loss = model.target_contrastive_loss(target_proj, text_proj, label_proj)
        label_loss = model.label_contrastive_loss(label_proj, text_proj, target_proj)

        # Calculate the overall loss
        total_loss = alpha * target_loss + beta * label_loss

        # Backpropagation and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

# Stance classification
# You can add your stance classification component here based on target_proj, text_proj, and label_proj


TypeError: ignored

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the projection layers
class ProjectionLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ProjectionLayer, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

# Define the Target-oriented contrastive loss
class TargetContrastiveLoss(nn.Module):
    def __init__(self, temperature):
        super(TargetContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, anchor, positive, negative):
        # Calculate the contrastive loss for target
        target_similarity = torch.nn.functional.cosine_similarity(anchor, positive, dim=-1) / self.temperature
        loss = -torch.log(torch.exp(target_similarity).sum() / (torch.exp(target_similarity).sum() + torch.exp(negative).sum()))
        return loss

# Define the Label-oriented contrastive loss
class LabelContrastiveLoss(nn.Module):
    def __init__(self, temperature):
        super(LabelContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, anchor, positive, negative):
        # Calculate the contrastive loss for label
        label_similarity = torch.nn.functional.cosine_similarity(anchor, positive, dim=-1) / self.temperature
        loss = -torch.log(torch.exp(label_similarity).sum() / (torch.exp(label_similarity).sum() + torch.exp(negative).sum()))
        return loss

# Define your StanceModel class
class StanceModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, temperature):
        super(StanceModel, self).__init__()
        self.target_projection = ProjectionLayer(input_dim, hidden_dim, output_dim)
        self.text_projection = ProjectionLayer(input_dim, hidden_dim, output_dim)
        self.label_projection = ProjectionLayer(input_dim, hidden_dim, output_dim)
        self.target_contrastive_loss = TargetContrastiveLoss(temperature)
        self.label_contrastive_loss = LabelContrastiveLoss(temperature)

    def forward(self, target, text, label):
        target_projection = self.target_projection(target)
        text_projection = self.text_projection(text)
        label_projection = self.label_projection(label)

        return target_projection, text_projection, label_projection

# Instantiate the StanceModel, optimizer, and contrastive losses
input_dim = 128  # Replace with the actual input dimension
hidden_dim = 64  # Replace with your desired hidden dimension
output_dim = 32  # Replace with your desired output dimension
temperature = 0.07  # Replace with your desired temperature

model = StanceModel(input_dim, hidden_dim, output_dim, temperature)
optimizer = optim.Adam(model.parameters(), lr=0.01)

from torch.utils.data import DataLoader, TensorDataset

# Example data (replace with your actual data)
targets_data = torch.randn(100, input_dim)  # Replace with your target data
texts_data = torch.randn(100, input_dim)  # Replace with your text data
labels_data = torch.randn(100, input_dim)  # Replace with your label data

# Create a TensorDataset
dataset = TensorDataset(targets_data, texts_data, labels_data)

# Define batch size
batch_size = 32  # Adjust the batch size as needed

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# Training loop
num_epochs = 10  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
     total_loss = 0.0
     correct = 0
     total = 0
     for batch in dataloader:
          targets, texts, labels = batch

          # Forward pass
          target_proj, text_proj, label_proj = model(targets, texts, labels)

          # Calculate target contrastive loss and label contrastive loss
          target_loss = model.target_contrastive_loss(target_proj, text_proj, label_proj)
          label_loss = model.label_contrastive_loss(label_proj, text_proj, target_proj)

          # Calculate the overall loss
          alpha = 0.5  # Adjust these values as needed
          beta = 0.5
          total_loss = alpha * target_loss + beta * label_loss

          # Backpropagation and optimization
          optimizer.zero_grad()
          total_loss.backward()
          optimizer.step()
          # _, predicted = target_proj.max(1)
          # total += labels.size(0)
          # correct += (predicted == labels).sum().item()

          epoch_loss = total_loss / len(dataloader)
          print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')





Epoch 1/10, Loss: 0.1423
Epoch 1/10, Loss: 0.0160
Epoch 1/10, Loss: 0.0034
Epoch 1/10, Loss: 0.0034
Epoch 2/10, Loss: 0.0007
Epoch 2/10, Loss: 0.0006
Epoch 2/10, Loss: 0.0008
Epoch 2/10, Loss: 0.0004
Epoch 3/10, Loss: 0.0007
Epoch 3/10, Loss: 0.0011
Epoch 3/10, Loss: 0.0006
Epoch 3/10, Loss: 0.0004
Epoch 4/10, Loss: 0.0006
Epoch 4/10, Loss: 0.0007
Epoch 4/10, Loss: 0.0006
Epoch 4/10, Loss: 0.0041
Epoch 5/10, Loss: 0.0008
Epoch 5/10, Loss: 0.0008
Epoch 5/10, Loss: 0.0003
Epoch 5/10, Loss: 0.0001
Epoch 6/10, Loss: 0.0004
Epoch 6/10, Loss: 0.0005
Epoch 6/10, Loss: 0.0006
Epoch 6/10, Loss: 0.0002
Epoch 7/10, Loss: 0.0005
Epoch 7/10, Loss: 0.0003
Epoch 7/10, Loss: 0.0003
Epoch 7/10, Loss: 0.0028
Epoch 8/10, Loss: 0.0004
Epoch 8/10, Loss: 0.0004
Epoch 8/10, Loss: 0.0003
Epoch 8/10, Loss: 0.0002
Epoch 9/10, Loss: 0.0002
Epoch 9/10, Loss: 0.0002
Epoch 9/10, Loss: 0.0005
Epoch 9/10, Loss: 0.0001
Epoch 10/10, Loss: 0.0001
Epoch 10/10, Loss: 0.0003
Epoch 10/10, Loss: 0.0004
Epoch 10/10, Loss: 0.0

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split


data = pd.read_excel('la_train.xlsx')

text_inputs = data['text']
labels = data['sentiment_label']


train_text, val_text, train_labels, val_labels = train_test_split(text_inputs, labels, test_size=0.2, random_state=42)

print("Training set shape - text:", train_text.shape)
print("Validation set shape - text:", val_text.shape)
print("Training set shape - labels:", train_labels.shape)
print("Validation set shape - labels:", val_labels.shape)


Training set shape - text: (3149,)
Validation set shape - text: (788,)
Training set shape - labels: (3149,)
Validation set shape - labels: (788,)
